# Рекомендация тарифов

# Описание проекта
Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Найдем модель с максимально большим значением accuracy.

### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
-  is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

import sklearn.metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

### Загружаем файл

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('...')

### Ознакамливаемся с данными

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Смотрим тип данных и пропуски

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Разбиваем данные на признаки и целевой признак

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

### Проверяем полученные таблицы

In [6]:
features.head()

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


In [7]:
target.head()

0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

### Разделяем данные на выборки

In [8]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.25, random_state=12345)

In [9]:
features_test, features_val, target_test, target_val = train_test_split(
    features_test, target_test, test_size=.5, random_state=12345)

In [10]:
print(features_train.shape)
print(features_test.shape)
print(features_val.shape)
print(target_train.shape)
print(target_test.shape)
print(target_val.shape)

(2410, 4)
(402, 4)
(402, 4)
(2410,)
(402,)
(402,)


Получаем следующие выборки: обучающая (75%), валидационная (12.5%) и тестовая (12.5%).

## Исследуем модели

### Решающее дерево

In [11]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for tree_depth in range(1, 4):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=tree_depth)
    tree_model.fit(features_train, target_train)
    tree_predictions_val = tree_model.predict(features_val)
    tree_result = accuracy_score(target_val, tree_predictions_val)
    if tree_result > best_tree_result:
        best_tree_model = tree_model
        best_tree_result = tree_result
        best_tree_depth = tree_depth
print("Accuracy наилучшей модели на валидационной выборке:", best_tree_result, "Глубина дерева:", best_tree_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7786069651741293 Глубина дерева: 3


### Случайный лес

In [12]:
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for forest_est in range(10, 51, 10):
    for forest_depth in range (1, 11):
        forest_model = RandomForestClassifier(random_state=12345, n_estimators=forest_est, max_depth=forest_depth)
        forest_model.fit(features_train, target_train)
        forest_predictions_val = forest_model.predict(features_val)
        forest_result = accuracy_score(target_val, forest_predictions_val)
        if forest_result > best_forest_result:
            best_forest_model = forest_model
            best_forest_result = forest_result
            best_forest_est = forest_est
            best_forest_depth = forest_depth

print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result, 
      "Количество деревьев:", best_forest_est, "Максимальная глубина:", best_forest_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8109452736318408 Количество деревьев: 30 Максимальная глубина: 9


- Была исследована эффективность 2-х моделей: решающее дерево и случайный лес.
- Для решающего дерева были проанализированы глубины с 1 до 5, глубина 3 имела максимальный accuracy.
- Для случайного леса были проанализированы десятки деревьев от 10 до 50 и глубины с 1 до 10. Самым эффективным оказалось 30 деревьев с максимальной глубиной равной 9.
- Из двух моделей случайный лес показал большую эффективность, accuracy равнялся ~81%.

## Проверяем модель на тестовой выборке

In [13]:
final_forest_model = RandomForestClassifier(random_state=12345, n_estimators=30, max_depth=9)
final_forest_model.fit (features_train, target_train)
final_forest_prediction = final_forest_model.predict(features_test)
final_forest_result = accuracy_score(target_test, final_forest_prediction)
print('Accuracy наилучшей модели на тестовой выборке:', final_forest_result)

Accuracy наилучшей модели на тестовой выборке: 0.8084577114427861


## Проверяем модели на адекватность

### Проверим модель на адекватность при сравнении с DummyClassifier

In [14]:
dummy_clf = DummyClassifier(random_state=12345, strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_prediction = dummy_clf.predict(features_test)
dummy_score = accuracy_score(target_test, dummy_prediction)

### Создадим матрицу ошибок для бинарной классификации

#### RandomForestClassifier

In [15]:
confusion_matrix = sklearn.metrics.confusion_matrix(target_test, final_forest_prediction)
confusion_matrix = np.flip(confusion_matrix)
confusion_matrix = pd.DataFrame(confusion_matrix, columns = ['+Positive', '-Negative'])
confusion_matrix.set_index(pd.Index(['+Positive', '-Negative']))

,+Positive,-Negative
+Positive,63,59
-Negative,18,262


In [16]:
true_positive = 63 / len(final_forest_prediction) * 100
false_negative = 59 / len(final_forest_prediction) * 100
false_positive = 18 / len(final_forest_prediction) * 100
true_negative = 262 / len(final_forest_prediction) * 100

print(true_positive, false_negative, false_positive, true_negative)

15.671641791044777 14.676616915422885 4.477611940298507 65.17412935323384


64 раз, ~16% модель правильно классифицировала Positive как Positive.

58 раз, ~14% модель неправильно классифицировала Positive как Negative.

18 раза, ~5% модель неправильно классифицировала Negative как Positive.

262 раз, ~65% модель правильно классифицировала Negative как Negative.

#### DummyClassifier

In [17]:
confusion_matrix = sklearn.metrics.confusion_matrix(target_test, dummy_prediction)
confusion_matrix = np.flip(confusion_matrix)
confusion_matrix = pd.DataFrame(confusion_matrix, columns = ['+Positive', '-Negative'])
confusion_matrix.set_index(pd.Index(['+Positive', '-Negative']))

,+Positive,-Negative
+Positive,0,122
-Negative,0,280


Видим, что DummyClassifier фиксирует все ответы, аналогично самым популярным, в нашем случае - 0 или негативными.

### Проверим Accuracy

#### RandomForestClassifier

In [18]:
final_forest_result

0.8084577114427861

Модель показывает ~81% точности предсказаний.

#### DummyClassifier

In [19]:
dummy_score

0.6965174129353234

DummyClassifier показывает ~70% точности предсказаний.

### Проверим Precision

#### RandomForestClassifier

In [20]:
precision = sklearn.metrics.precision_score(target_test, final_forest_prediction)
print(precision)

0.7777777777777778


Модель показывает ~78% точности положительных предсказаний.
Если модель называет целевой признак положительным, то с вероятностью ~78% это так и есть.

#### DummyClassifier

In [21]:
dummy_precision = sklearn.metrics.precision_score(target_test, dummy_prediction)
print(dummy_precision)

0.0


/Users/sergejvorobev/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Проверим Recall

#### RandomForestClassifier

In [22]:
recall = sklearn.metrics.recall_score(target_test, final_forest_prediction)
print(recall)

0.5163934426229508


Модель показывает ~52% точности при поиске положительных значений.
Модель находит лишь ~52% всех положительных значений. Зная это, мы можем понять, что предсказания по количеству переходов на новый тариф будут ниже, чем фактические показатели.

#### DummyClassifier

In [23]:
dummy_recall = sklearn.metrics.recall_score(target_test, dummy_prediction)
print(dummy_recall)

0.0


# Выводы
- Была исследована эффективность 2-х моделей: решающее дерево и случайный лес.
- Для решающего дерева были проанализированы глубины с 1 до 5, глубина 3 имела максимальный accuracy.
- Для случайного леса были проанализированы десятки деревьев от 10 до 50 и глубины с 1 до 10. Самым эффективным оказалось 30 деревьев с максимальной глубиной равной 9.
- Из двух моделей случайный лес показал большую эффективность, accuracy равнялся ~81%.

Данная модель имела следующие показатели:
- 64 раз, ~16% модель правильно классифицировала Positive как Positive.
- 58 раз, ~14% модель неправильно классифицировала Positive как Negative.
- 18 раза, ~5% модель неправильно классифицировала Negative как Positive.
- 262 раз, ~65% модель правильно классифицировала Negative как Negative.
- Видим, что DummyClassifier фиксирует все ответы, аналогично самым популярным, в нашем случае - 0 или негативными.
- Модель показывает ~81% точности предсказаний.
- DummyClassifier показывает ~70% точности предсказаний.
- Модель показывает ~78% точности положительных предсказаний.
- Если модель называет целевой признак положительным, то с вероятностью ~78% это так и есть.
- Модель показывает ~52% точности при поиске положительных значений.
- Модель находит лишь ~52% всех положительных значений. Зная это, мы можем понять, что предсказания по количеству переходов на новый тариф будут ниже, чем фактические показатели.